In [1]:
# !pip install opencv-python
# !pip install numpy
# !pip install matplotlib
# !pip install scikit-learn
# !pip install -U scikit-learn scipy matplotlib

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

dragging = False
x, y = 250, 150

def find_histogram(clt):
    """
    create a histogram with k clusters
    :param: clt
    :return:hist
    """
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    hist = hist.astype("float")
    hist /= hist.sum()

    return hist

def plot_colors2(hist, centroids):
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0

    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                      color.astype("uint8").tolist(), -1)
        startX = endX

    # return the bar chart
    return bar

def resize(frame):
    return cv2.resize(frame, (800, 600))

def drag_rectangle(event, mouseX, mouseY, flags, param):
    global x, y, dragging
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if x <= mouseX <= x+w and y <= mouseY <= y+h:  # Check if the cursor is inside the rectangle
            dragging = True
    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False
    elif event == cv2.EVENT_MOUSEMOVE and dragging:
        x = mouseX - w//2
        y = mouseY - h//2

In [3]:
webcam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', drag_rectangle)
ret, imageFrame = webcam.read()
while(ret):
    ret, imageFrame = webcam.read()

    if ret==True:        
        x, y, w, h = 250, 150, 350, 200
        imageFrame = cv2.rectangle(imageFrame,(x,y),(x+w,y+h),(0,255,0),2)
        roi = imageFrame[y:y+h, x:x+w].copy()
            
        img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        img = img.reshape((img.shape[0] * img.shape[1],3))
        clt = KMeans(n_clusters=4)
        clt.fit(img)

        hist = find_histogram(clt)
        bar = plot_colors2(hist, clt.cluster_centers_)
        bar = cv2.cvtColor(bar, cv2.COLOR_RGB2BGR)

        cv2.imshow('Frame', resize(imageFrame))
        cv2.imshow('Dominant Color', bar)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

webcam.release()
cv2.destroyAllWindows()
